# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Y. Wang  ->  Y. Wang  |  ['Y. Wang']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
F. Walter  ->  F. Walter  |  ['F. Walter']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Arxiv has 48 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2505.04688
extracting tarball to tmp_2505.04688...

 done.


Found 31 bibliographic references in tmp_2505.04688/blank.bbl.
Issues with the citations
syntax error in line 53: '=' expected
Retrieving document from  https://arxiv.org/e-print/2505.04707
extracting tarball to tmp_2505.04707...

 done.


K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
F. Walter  ->  F. Walter  |  ['F. Walter']


Found 79 bibliographic references in tmp_2505.04707/main.bbl.
Issues with the citations
syntax error in line 424: '=' expected
Retrieving document from  https://arxiv.org/e-print/2505.05021
extracting tarball to tmp_2505.05021... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.04688-b31b1b.svg)](https://arxiv.org/abs/2505.04688) | **Euclid preparation. The impact of redshift interlopers on the two-point correlation function analysis**  |
|| E. Collaboration, et al. -- incl., <mark>Y. Wang</mark>, <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-05-09*|
|*Comments*| *27 pages, 22 figures, submitted to A&A*|
|**Abstract**|            The Euclid survey aims to measure the spectroscopic redshift of emission-line galaxies by identifying the H$\,{\alpha}$ line in their slitless spectra. This method is sensitive to the signal-to-noise ratio of the line, as noise fluctuations or other strong emission lines can be misidentified as H$\,{\alpha}$, depending on redshift. These effects lead to catastrophic redshift errors and the inclusion of interlopers in the sample. We forecast the impact of such redshift errors on galaxy clustering measurements. In particular, we study the effect of interloper contamination on the two-point correlation function (2PCF), the growth rate of structures, and the Alcock-Paczynski (AP) parameters. We analyze 1000 synthetic spectroscopic catalogues, the EuclidLargeMocks, designed to match the area and selection function of the Data Release 1 (DR1) sample. We estimate the 2PCF of the contaminated catalogues, isolating contributions from correctly identified galaxies and from interlopers. We explore different models with increasing complexity to describe the measured 2PCF at fixed cosmology. Finally, we perform a cosmological inference and evaluate the systematic error on the inferred $f\sigma_8$, $\alpha_{\parallel}$ and $\alpha_{\perp}$ values associated with different models. Our results demonstrate that a minimal modelling approach, which only accounts for an attenuation of the clustering signal regardless of the type of contaminants, is sufficient to recover the correct values of $f\sigma_8$, $\alpha_{\parallel}$, and $\alpha_{\perp}$ at DR1. The accuracy and precision of the estimated AP parameters are largely insensitive to the presence of interlopers. The adoption of a minimal model induces a 1%-3% systematic error on the growth rate of structure estimation, depending on the redshift. However, this error remains smaller than the statistical error expected for the Euclid DR1 analysis.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.04707-b31b1b.svg)](https://arxiv.org/abs/2505.04707) | **Physical Conditions of the Ionized Superwind in NGC 253 with VLT/MUSE**  |
|| S. A. Cronin, et al. -- incl., <mark>K. Kreckel</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2025-05-09*|
|*Comments*| *21 pages, 14 figures, submitted to ApJ, incorporated feedback from referee report*|
|**Abstract**|            We present an analysis of the H$\alpha$-emitting ionized gas in the warm phase of the NGC 253 outflow using integral field spectroscopy from the Multi Unit Spectroscopic Explorer (MUSE). In each spaxel, we decompose H$\alpha$, [N II], and [S II] emission lines into a system of up to 3 Gaussian components, accounting for the velocity contributions due to the disk and both intercepted walls of an outflow cone. In the approaching southern lobe of the outflow, we find maximum deprojected outflow velocities down to ~ -500 km/s. Velocity gradients of this outflowing gas range from ~ -350 to -550 km/s/kpc with increasing distance from the nucleus. Additionally, [N II]/H$\alpha$ and [S II]/H$\alpha$ integrated line ratios are suggestive of shocks as the dominant ionization source throughout the wind. Electron densities, inferred from the [S II] doublet, peak at 2100 cm$^{-3}$ near the nucleus and reach $\lesssim 50 $cm$^{-3}$ in the wind. Finally, at an uncertainty of 0.3 dex on the inferred mass of $4\times10^{5}$ M$_{\odot}$, the mass-outflow rate of the H$\alpha$-emitting gas in the southern outflow lobe is ~ 0.4 M$_{\odot}$/year. This yields a mass-loading factor of $\eta$ ~ 0.1 and a ~ 2% starburst energy efficiency.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.05021-b31b1b.svg)](https://arxiv.org/abs/2505.05021) | **Searching for pulsars in Globular Clusters with the Fast Fold Algorithm and a new pulsar discovered in M13**  |
|| Y. Li, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-05-09*|
|*Comments*| **|
|**Abstract**|            We employed the Fast Folding Algorithm (FFA) on L-Band Globular Cluster (GC) observations taken with Five-hundred-meter Aperture Spherical radio Telescope (FAST) to search for new pulsars, especially those with a long rotational period. We conducted a search across 16 GCs that collectively host 93 known pulsars, as well as 14 GCs that do not contain any known pulsars. The majority of these known pulsars were successfully re-detected in our survey. The few non-detections could be attributed to the high accelerations of these pulsars. Additionally, we have discovered a new binary millisecond pulsar, namely M13I (or PSR J1641+3627I) in GC M13 (or NGC 6205), and obtained its phase-coherent timing solution using observations spanning 6 years. M13I has a spin period of 6.37 ms, and an orbital period of 18.23 days. The eccentricity of the binary orbit is 0.064, with a companion mass range of approximately 0.45 to 1.37 M$_{\odot}$. The orbital properties of M13I are remarkably different from those of the other known pulsars in M13, indicating that this pulsar has undergone a different evolutionary path compared to the rest.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2505.04688/./Figures/z1z3_params_comparison_fs8bs8_subset_paper_default.png', 'tmp_2505.04688/./Figures/from2D_overlpapped_contributes_z1z3.png', 'tmp_2505.04688/./Figures/z1_fs8bs8_paper_default.png', 'tmp_2505.04688/./Figures/z3_fs8bs8_paper_default.png']
copying  tmp_2505.04688/./Figures/z1z3_params_comparison_fs8bs8_subset_paper_default.png to _build/html/
copying  tmp_2505.04688/./Figures/from2D_overlpapped_contributes_z1z3.png to _build/html/
copying  tmp_2505.04688/./Figures/z1_fs8bs8_paper_default.png to _build/html/
copying  tmp_2505.04688/./Figures/z3_fs8bs8_paper_default.png to _build/html/
exported in  _build/html/2505.04688.md
    + _build/html/tmp_2505.04688/./Figures/z1z3_params_comparison_fs8bs8_subset_paper_default.png
    + _build/html/tmp_2505.04688/./Figures/from2D_overlpapped_contributes_z1z3.png
    + _build/html/tmp_2505.04688/./Figures/z1_fs8bs8_paper_default.png
    + _build/html/tmp_2505.04688/./Figures/z3_fs8bs8_paper_default.png
found fig

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\xic}{\xi_{\rm m}}$
$\newcommand{\xit}{\xi_{\rm cc}}$
$\newcommand{\xin}{\xi_{\rm nn}}$
$\newcommand{\xil}{\xi_{\ell\ell}}$
$\newcommand{\xitn}{\xi_{\rm cn}}$
$\newcommand{\xitl}{\xi_{\rm c\ell}}$
$\newcommand{\xiln}{\xi_{\ell \rm n}}$
$\newcommand{\rt}{R_{\rm c}}$
$\newcommand{\rn}{R_{\rm n}}$
$\newcommand{\rl}{R_{\ell}}$
$\newcommand{\rc}{R_{\rm m}}$
$\newcommand{\fc}{f_{\rm tot}}$
$\newcommand{\bx}{\mathbf{x}}$
$\newcommand{\br}{\mathbf{r}}$
$\newcommand{\orcid}[1]$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# $\Euclid$ preparation: The impact of redshift interlopers on the two-point correlation function analysis

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.04688-b31b1b.svg)](https://arxiv.org/abs/2505.04688)<mark>Appeared on: 2025-05-09</mark> -  _27 pages, 22 figures, submitted to A&A_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>Y. Wang</mark>, <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The $\Euclid$ galaxy survey is designed to measure the spectroscopic redshift of emission-line galaxies (ELG) by identifying the $\ha$ emission line in their slitless spectra. The efficacy of this approach depends crucially on the signal-to-noise ratio (SNR) of the line, as sometimes noise fluctuations in the spectrum continuum can be misidentified as $\ha$ . In addition, other genuine strong emission lines can be mistaken for $\ha$ , depending on the redshift of the source. Both effects lead to ambiguities in the redshift measurement, potentially resulting in catastrophic redshift errors and the inclusion of `interloper galaxies in the sample. This paper forecasts the impact on the galaxy clustering analysis of the expected redshift errors in the $\Euclid$ spectroscopic sample. Specifically, it investigates the effect of the redshift interloper contamination on the galaxy two-point correlation function (2PCF) and, in turn, on the inferred growth rate of structure $\(f \sigma_8\)$ and Alcock--Paczynski (AP) parameters, $\(\alpha_\parallel\)$ and $\(\alpha_\perp\)$ . This work is based on the analysis of $\num{1000}$ synthetic spectroscopic catalogues, the _EuclidLargeMocks_ , which mimic the area and selection function of the $\Euclid$ Data Release 1 (DR1) sample. We estimate the 2PCF of contaminated catalogues and separate the different contributions, particularly those coming from galaxies with correct measured redshift and from contaminants. We explore different models with increasing complexity to describe the measured 2PCF at fixed cosmology, with the aim of identifying the most efficient model in reproducing the data. Finally, we perform a cosmological inference and evaluate the systematic error on the inferred $\(f \sigma_8\)$ , $\(\alpha_\parallel\)$ and $\(\alpha_\perp\)$ values associated with different models. Our results demonstrate that a minimal modelling approach, which only accounts for an attenuation of the clustering signal regardless of the type of contaminants, is sufficient to recover the correct values of $\(f \sigma_8\)$ and $\(\alpha_\parallel\)$ , $\(\alpha_\perp\)$ at DR1. The accuracy and precision of the estimated AP parameters are largely insensitive to the presence of interlopers. The adoption of a minimal modelling induces a 1 \% --3 \% systematic error on the growth rate of structure estimation, depending on the considered redshift. However, this error remains smaller than the statistical error expected for the $\Euclid$ DR1 analysis.

</div>

<div id="div_fig1">

<img src="tmp_2505.04688/./Figures/z1z3_params_comparison_fs8bs8_subset_paper_default.png" alt="Fig18" width="100%"/>

**Figure 18. -** $f\sigma_8$ and $b\sigma_8$ values and $1 \sigma$ uncertainties for all tests in the two reference redshift bins. They grey band corresponds to $1 \sigma$ around the values inferred in the reference case. (*fig:errorbars-parameters*)

</div>
<div id="div_fig2">

<img src="tmp_2505.04688/./Figures/from2D_overlpapped_contributes_z1z3.png" alt="Fig16" width="100%"/>

**Figure 16. -** Monopole, quadrupole and hexadecapole moments of all terms in Eq. \eqref{eq:contam-2pcf-RR} averaged over all mock catalogues for $z \in \left[0.9, 1.1\right]$(_left_) and $z \in \left[1.3, 1.5\right]$(_right_). All terms comprise the correlation function and the corresponding prefactor. To simplify the visualization of all terms, the rightmost column of each panel shows a zoom-in on the smallest contributions in the corresponding redshift bin. (*fig:multipoles-z1-z3*)

</div>
<div id="div_fig3">

<img src="tmp_2505.04688/./Figures/z1_fs8bs8_paper_default.png" alt="Fig7.1" width="50%"/><img src="tmp_2505.04688/./Figures/z3_fs8bs8_paper_default.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** Contour plots for $f\sigma_8$ and $b\sigma_8$ derived from three representative tests in $z \in [0.9, 1.1]$(_top_) and $z \in [1.3, 1.5]$(_bottom_). The dashed grey lines indicate the mean values of the reference case posterior distribution. (*fig:fs8b_contour_all*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.04688"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Physical Conditions of the Ionized Superwind in NGC 253 with VLT/MUSE

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.04707-b31b1b.svg)](https://arxiv.org/abs/2505.04707)<mark>Appeared on: 2025-05-09</mark> -  _21 pages, 14 figures, submitted to ApJ, incorporated feedback from referee report_

</div>
<div id="authors">

S. A. Cronin, et al. -- incl., <mark>K. Kreckel</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present an analysis of the $\ha$ -emitting ionized gas in the warm phase of the NGC 253 outflow using integral field spectroscopy from the Multi Unit Spectroscopic Explorer (MUSE). In each spaxel, we decompose $\ha$ , $\nii$ , and $\sii$ emission lines into a system of up to 3 Gaussian components, accounting for the velocity contributions due to the disk and both intercepted walls of an outflow cone. In the approaching southern lobe of the outflow, we find maximum deprojected outflow velocities down to $\sim -500$  $\kms$ . Velocity gradients of this outflowing gas range from $\sim -350$ to $-550$  $\kms$ kpc $^{-1}$ with increasing distance from the nucleus.  Additionally, $\nii$ / $\ha$ and $\sii$ / $\ha$ integrated line ratios are suggestive of shocks as the dominant ionization source throughout the wind. Electron densities, inferred from the $\sii$ doublet, peak at $2100$ cm $^{-3}$ near the nucleus and reach $\lesssim 50$ cm $^{-3}$ in the wind. Finally, at an uncertainty of $0.3$ dex on the inferred mass of $4\times10^{5}$  $\msun$ , the mass-outflow rate of the $\ha$ -emitting gas in the southern outflow lobe is $\sim 0.4$  $\msunperyear$ . This yields a mass-loading factor of $\eta \sim 0.1$ and a $\sim 2\%$ starburst energy efficiency.

</div>

<div id="div_fig1">

<img src="tmp_2505.04707/./deproj_diagram.png" alt="Fig8" width="100%"/>

**Figure 8. -** Geometry of a cone with inclination $i$ and semi-opening angle $\alpha$ measured with respect to its rotational axis ($\hat{a}$; dashed line). (a) As a function of $\varphi$(the azimuthal angle around the cone axis), $\vec{v}_{\rm deproj}$ traces the wall of the cone, and the projected velocity vector $\vec{v}_{\rm proj}$(solid black line) lies in the $XY$ plane. Here, $\vec{v}_{\rm deproj}$ points toward the observer at $\varphi = 270^\circ$(left black point), in which the velocity has a maximum projection factor ($|\cos(i - \alpha)|$) toward the observer. The maximum projection factor away from the observer is then $|\cos(i + \alpha)|$ at $\varphi = 90^\circ$(right black point). (b) A 3D cone in the $XYZ$ plane with the observer at $X = 0$. (c) The same cone as (a), but noting the angle projected from the sky plane. For $i = 78^{\circ}$, this angle is small ($12^{\circ}$), which is why most lines-of-sight do not detect the back wall of the cone. (*fig:deproj*)

</div>
<div id="div_fig2">

<img src="tmp_2505.04707/./se_projvel_fwhm.png" alt="Fig9" width="100%"/>

**Figure 9. -** Kinematics of the southern outflow lobe in \ngc. _Top:_ Projected \niibwvl velocities of the main outflow cone ("Blue Outflow"), additional redshifted outflow components ("Redshifted Outflow") and disk components ("Disk"). The average projected velocities of the southern outflow cone are between $\sim -150$ and $-250$ \kms. In the bluest region of the Blue Outflow map, velocities reach $\sim -350$ \kms. Redshifted components in both outflow maps may be caused by confusion with the intervening disk. _Bottom:_ Intrinsic FWHM calculated by subtracting in quadrature the instrumental resolution ($100$ \kms) from the measured FWHM. The average Blue Outflow linewidth map is $\sim 250$ \kms. Broad linewidths may indicate that the ionized gas is filling the outflow cone.  _ Both:_ JWST MIRI F770W contours highlight the nucleus, spiral arms, and outflow filaments. The white $\times$ marks the galaxy center. (*fig:kinematics_se*)

</div>
<div id="div_fig3">

<img src="tmp_2505.04707/./se_flux_matsubayashi_regions.png" alt="Fig7" width="100%"/>

**Figure 7. -** Integrated flux of \ha, \niiawvl$+$\niibwvl, and \siiawvl$+$\siibwvl in each component in the SE cube. The white $\times$ marks the galaxy center. All emission lines are brightest in Hii regions and the starbursting nucleus. The emission is not uniformly bright throughout the Blue Outflow, indicating that the outflow cone is clumpy. (*fig:flux_se*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.04707"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

482  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
